# Run z-brains analyses for 3T and 7T cases

In [11]:
import pandas as pd
import os, sys, subprocess
import importlib

# import utilities
utilities_path = os.path.abspath(os.path.join(os.getcwd(), "/host/verges/tank/data/daniel", "Utils"))
sys.path.append(utilities_path) # path to Utilities
import id, gen, util

In [20]:
importlib.reload(id)

<module 'id' from '/host/verges/tank/data/daniel/Utils/id.py'>

In [3]:
# zBrain parameters

# IDs: should be in long format (all IDs to be used should be in same column (regardless of if it is 3T or 7T ID))
dir_IDs="/host/verges/tank/data/daniel/3T7T/z/data/pt/IDs-3T7T-long_test_11Mar2025.csv" # long format IDs

# size of kernels for smoothing (cortex: ctx; hippocampus: hip). 
# NOTE: Both lists must be of same length.
smooth_ctx=[2,5,10] # try with 2,5,10
smooth_hip=[1,2,5] # try with 1,2,5

# surface
res="high" # low (cotrex: 5K, hippocampus: 2mm), high (ctx: 32k, hipp: 0.5mm), all (both high and low)

# Output: path and file name
dir_out="/host/verges/tank/data/daniel/3T7T/z/outputs"
out_name="zb_3-7comp" # will create folders with this name

# path to z-brains software
# zbrains_sh_path="/host/verges/tank/data/daniel/z-brains/zbrains"

# needs to be in BIDs directory
zbrains_sh_path = "/host/verges/tank/data/daniel/3T7T/z/code/run_zb/printArgs.sh"

In [ ]:
# dictionary item for each study

# Note: zBrains only currently supports one 'ctrl_ptrn', not a list


# directories need to be relative except for root
MICs = {
    "name" : "MICs",
    "ctrl_ptrn" : ["HC"],
    "pt_ptrn" : ["PX"],
    "dir_root" : "/data/mica3/BIDS_MICs",
    "dir_raw" : "/data/mica3/BIDS_MICs/rawdata",
    "dir_deriv" : "/data/mica3/BIDS_MICs/derivatives",
    "dir_mp" : "/data/mica3/BIDS_MICs/derivatives/micapipe_v0.2.0",
    "dir_hu" : "/data/mica3/BIDS_MICs/derivatives/hippunfold_v1.3.0",
    "zb" : dir_out + "/" + out_name + "_MICs"
}

PNI = {
    "name" : "PNI",
    "ctrl_ptrn" : ["PNC","Pilot"],
    "pt_ptrn" : ["PNE"],
    "dir_root" : "/data/mica3/BIDS_PNI",
    "dir_raw" : "data/mica3/BIDS_PNI/rawdata",
    "dir_deriv" : "data/mica3/BIDS_PNI/derivatives",
    "dir_mp" : "data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0",
    "dir_hu" : "data/mica3/BIDS_PNI/derivatives/hippunfold_v1.3.0",
    "zb" : dir_out + "/" + out_name + "_PNI"
}

studies = [MICs, PNI]

In [5]:
# Convert IDs to format accepted by zBrains (add sub-, ses-)

# seperate IDs by study - pass seperate IDs csv to zBrains
# extract patterns
ptrns = []
for study in studies:
    ptrns.append((study["ctrl_ptrn"]) + study["pt_ptrn"])

out = dir_out+"/tmp"
split_paths = id.split(dir_IDs, col="ID", patterns=ptrns, pth_out=out) # split_paths holds paths to split files

# print(split_paths)
# length of split_paths should be equal to the number of studies

if len(split_paths) != len(studies):
    print("[main] Error: Number of split files does not match number of studies.")
    print("\t Check  that the attributes 'ctrl_ptrn' and 'pt_ptrn' are properly defined in all study dictionaries.")
    sys.exit(1)

[split] Saved: /host/verges/tank/data/daniel/3T7T/z/outputs/tmp/split_ID_1.csv (4 rows)HC: 2, PX: 2
[split] Saved: /host/verges/tank/data/daniel/3T7T/z/outputs/tmp/split_ID_2.csv (5 rows)PNC: 2, PNE: 2, Pilot: 1


In [15]:
# Call zbrains - can be parallelized
for i in [1,len(studies)]:
    df_path = split_paths[i-1]
    study = studies[i-1]
    print("\n=====================================")
    print(study["name"])
    #--control_prefix and define according to study

    # run z-brains with each smoothing kernel - can run cortex and hippocampus seperately and in parallel
    for ctx, hip in zip(smooth_ctx, smooth_hip):
        print(f"Smoothing: ctx={ctx}, hip={hip}")
        
        # call via bash, pass appropriate arguments
        result = util.runZBrain(df_path, study, ctx, hip, res, dir_software=zbrains_sh_path)

        # Print zBrains run output
        if result.returncode != 0:
            print(f"Command failed with return code {result.returncode}")

        print("STDOUT:")
        for line in result.stdout.splitlines():
            print(f"\t{line}")

        if result.stderr:
            print("STDERR:")
            for line in result.stderr.splitlines():
                print(f"\t{line}")
        


MICs
Smoothing: ctx=2, hip=1
STDOUT:
	[bash] Passed arguments:
		1 	 /host/verges/tank/data/daniel/3T7T/z/outputs/tmp/split_ID_1.csv
		2 	 /data/mica3/BIDS_MICs
		3 	 /data/mica3/BIDS_MICs/derivatives/micapipe_v0.2.0
		4 	 /data/mica3/BIDS_MICs/derivatives/hippunfold_v1.3.0
		5 	 /host/verges/tank/data/daniel/3T7T/z/outputs/zb_3-7comp_MICs
		6 	 ['HC']
		7 	 2
		8 	 1
		9 	 high
Smoothing: ctx=5, hip=2
STDOUT:
	[bash] Passed arguments:
		1 	 /host/verges/tank/data/daniel/3T7T/z/outputs/tmp/split_ID_1.csv
		2 	 /data/mica3/BIDS_MICs
		3 	 /data/mica3/BIDS_MICs/derivatives/micapipe_v0.2.0
		4 	 /data/mica3/BIDS_MICs/derivatives/hippunfold_v1.3.0
		5 	 /host/verges/tank/data/daniel/3T7T/z/outputs/zb_3-7comp_MICs
		6 	 ['HC']
		7 	 5
		8 	 2
		9 	 high
Smoothing: ctx=10, hip=5
STDOUT:
	[bash] Passed arguments:
		1 	 /host/verges/tank/data/daniel/3T7T/z/outputs/tmp/split_ID_1.csv
		2 	 /data/mica3/BIDS_MICs
		3 	 /data/mica3/BIDS_MICs/derivatives/micapipe_v0.2.0
		4 	 /data/mica3/BIDS_MICs